In [76]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import pandas as pd
import os, glob

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/conorcorbin/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'mining-clinical-decisions' 
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


## UTI Labelling Cohort Project

**Objective**: The purpose of this notebook is to define a cohort and conduct basic exploratory-data-analysis (eda) for patients with suspected UTI. We plan to manually review a subset of these cases to find instances where the urine culture was negative but the patient needed antibiotics as well as instances where the urine culture was positive but the patient did not need antibiotics. We'll then use these expert labels to train phenotyping models that attempt to address infection status in a way that is more robust than simply relying on whether the urine culture was positive. 

**Cohort Description**: We include adult patients with a urine culture ordered at Stanford hospital and clinics. The urine culture must be the first urine culture ordered in a two week time period. We only include urine culture orders that also appear in lab results.  If an eventual model is triggered by an order for a urine culture, this may lead to biased estimates of performance because not all ordered urine cultures end up on lab results. 

**Table Schema**: The SQL query below produces a table with the following schema. 
* `anon_id`: patient identifier
* `pat_enc_csn_id_coded`: csn associated with urine culture order
* `order_proc_id_coded`: id associated with order for urine culture
* `order_time_jittered_utc`: time of urine culture order
* `ordering_mode`: either inpatient, outpatient, or null
* `was_positive`: 1 if urine culture was positive, 0 otherwise
* `had_other_pos_culture`: 1 if some other culture (from any anatomical site) ordered within a 48 hour time window centered at the urine culture order time was positive, 0 otherwise
* `ordered_antibiotics`: comma separated list of antibiotics ordered for the patient during the four hour window after the order of the urine culture
* `organisms`: comma separated list of organisms that grew from the urine culture 
* `other_pos_sites`: comma separated list of other cultures from all anatomical sites that were positive within 48 hour time window centered at urine culture order time
* `other_organisms`: comma separated list of organisms that grew from these other positive cultures


#### Other columns to add
* `wbc_count` : closest WBC count to urine culture order time
* `neutra` : closest absolute neutrophil count to urine culture order time
* `temperature` : closest temperature to urine culture order time
* `dx codes related to infection` : any dx codes related to bacterial infection
* `urine analysis results` : leukocytes, nitrite


In [77]:
%%bigquery df_uc_all
-- All urine culture orders at Stanford, order_id must be in lab_results
WITH urine_cultures AS (
    SELECT DISTINCT
        op.anon_id, op.pat_enc_csn_id_coded, op.order_proc_id_coded, op.order_time_jittered_utc, op.ordering_mode
    FROM 
        `mining-clinical-decisions.shc_core.order_proc` op
    INNER JOIN
        `mining-clinical-decisions.shc_core.lab_result` lr
    ON
        op.order_proc_id_coded = lr.order_id_coded
    WHERE
        op.order_type LIKE "Microbiology%"
        AND op.description LIKE "%URINE%"
),

-- Filter to adult only
adult_urine_cultures AS (
    SELECT DISTINCT
        uc.*
    FROM 
        urine_cultures uc
    INNER JOIN
        `shc_core.demographic` demo
    USING
        (anon_id)
    WHERE
        DATE_DIFF(CAST(uc.order_time_jittered_utc as DATE), demo.BIRTH_DATE_JITTERED, YEAR) >= 18
),

-- Must not have any urine culture orders (that go on to result in lab results) in prior two weeks
-- This finds urine cultures that do have other urine cultures orders in prior two weeks
order_in_prior_two_weeks AS (
    SELECT DISTINCT
         auc.order_proc_id_coded
    FROM 
        `mining-clinical-decisions.shc_core.order_proc` op
    INNER JOIN
        `mining-clinical-decisions.shc_core.lab_result` lr
    ON
        op.order_proc_id_coded = lr.order_id_coded
    INNER JOIN
        adult_urine_cultures auc 
    ON
        op.anon_id = auc.anon_id
    WHERE
        op.order_type LIKE "Microbiology%"
        AND op.description LIKE "%URINE%"
        AND auc.order_time_jittered_utc > op.order_time_jittered_utc
        AND TIMESTAMP_DIFF(auc.order_time_jittered_utc, op.order_time_jittered_utc, DAY) < 14
),

-- Set of antibiotics that show up in order_med (may not be totally all encompassing) 
include_abx AS (
    SELECT
        med_description
    FROM
        `mining-clinical-decisions.abx.abx_types` 
    WHERE
        is_include_abx = 1 OR is_oral = 1
),

-- Antibiotics ordered within 48 hour window of urine culture
ordered_abx AS (
    SELECT 
        auc.*, om.med_description 
    FROM 
        adult_urine_cultures auc
    INNER JOIN
        `mining-clinical-decisions.shc_core.order_med` om
    USING
        (anon_id)
    WHERE
        om.med_description IN (SELECT med_description FROM include_abx)
    AND 
        TIMESTAMP_DIFF(om.order_inst_utc, auc.order_time_jittered_utc, HOUR) BETWEEN -24 AND 24 
),

-- Remove urine cultures that have prior urine culture and get append list of antibiotics ordered in first 24 hours if any
included_urine_cultures AS (
    SELECT DISTINCT
        auc.*, oa.med_description
    FROM 
        adult_urine_cultures auc
    LEFT JOIN
        ordered_abx oa
    USING
        (order_proc_id_coded)
    WHERE 
        auc.order_proc_id_coded NOT IN (SELECT order_proc_id_coded  FROM order_in_prior_two_weeks)
),

-- Flag to indicated if the culture was positive, and resulting organisms
orders_that_were_positive AS (
    SELECT DISTINCT
        auc.order_proc_id_coded, 1 as was_positive, organism
    FROM
        included_urine_cultures auc
    INNER JOIN  
        (SELECT DISTINCT order_proc_id_coded, organism
        FROM `mining-clinical-decisions.shc_core.culture_sensitivity`) cs
    USING
        (order_proc_id_coded) 
),

--Other postive culture types ordered within window of urine culture order. window = -24 to 24 hours
orders_with_other_pos_cultures AS (
    SELECT DISTINCT
        auc.order_proc_id_coded, 1 as had_other_pos_culture, description other_pos_site, organism other_organism
    FROM
        included_urine_cultures auc
    INNER JOIN  
        `mining-clinical-decisions.shc_core.culture_sensitivity` cs
    USING
        (anon_id) 
    WHERE
        auc.order_proc_id_coded <> cs.order_proc_id_coded
    AND
        -- Some urine culture orders have two order ids where only one results, make sure we're not counting these.
        -- Likely not needed logic after forcing orders to appear in lab_results
        (auc.order_time_jittered_utc <> cs.order_time_jittered_utc OR cs.description NOT LIKE '%URINE%')
    AND
        TIMESTAMP_DIFF(cs.order_time_jittered_utc, auc.order_time_jittered_utc, HOUR)
    BETWEEN
        -24 AND 24
),

--Temporary building block that has medications and organisms in long form
orders_meds_and_bugs AS (
SELECT DISTINCT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, order_time_jittered_utc, ordering_mode, 
    CASE WHEN was_positive = 1 THEN 1 ELSE 0 END was_positive,
    CASE WHEN had_other_pos_culture = 1 THEN 1 ELSE 0 END had_other_pos_culture,
    med_description,
    organism,
    other_pos_site,
    other_organism
FROM 
    included_urine_cultures
LEFT JOIN 
    orders_with_other_pos_cultures
USING 
    (order_proc_id_coded)
LEFT JOIN 
    orders_that_were_positive
USING
    (order_proc_id_coded)
ORDER BY
    order_proc_id_coded
),

-- Collapse medication orders to order id of the urine culture
concatenated_meds AS (
    SELECT DISTINCT
        order_proc_id_coded,
        STRING_AGG(DISTINCT med_description ORDER BY med_description) ordered_antibiotics
    FROM
        orders_meds_and_bugs
    GROUP BY
        order_proc_id_coded
),

-- Collpase organisms to order id of the urine culture
concatenated_bugs AS (
    SELECT DISTINCT
        order_proc_id_coded,
        STRING_AGG(DISTINCT organism ORDER BY organism) organisms
    FROM
        orders_meds_and_bugs
    GROUP BY
        order_proc_id_coded
),

-- Collpase other positive sites
concatenated_sites AS (
    SELECT DISTINCT
        order_proc_id_coded,
        STRING_AGG(DISTINCT other_pos_site ORDER BY other_pos_site) other_pos_sites
    FROM
        orders_meds_and_bugs
    GROUP BY
        order_proc_id_coded
),

-- Collpase other organisms
concatenated_other_bugs AS (
    SELECT DISTINCT
        order_proc_id_coded,
        STRING_AGG(DISTINCT other_organism ORDER BY other_organism) other_organisms
    FROM
        orders_meds_and_bugs
    GROUP BY
        order_proc_id_coded
)

SELECT DISTINCT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, order_time_jittered_utc, ordering_mode, was_positive, had_other_pos_culture,
    ordered_antibiotics, organisms, other_pos_sites, other_organisms
FROM
    orders_meds_and_bugs
INNER JOIN 
    concatenated_meds
USING
    (order_proc_id_coded)
INNER JOIN 
    concatenated_bugs 
USING
    (order_proc_id_coded)
INNER JOIN 
    concatenated_sites  
USING
    (order_proc_id_coded)
INNER JOIN 
    concatenated_other_bugs  
USING
    (order_proc_id_coded)
ORDER BY 
    order_proc_id_coded

Executing query with job ID: f884c149-c3fd-4a1e-b935-6b45ec7dd3da
Query executing: 0.69s


ERROR:
 400 Resources exceeded during query execution: Not enough resources for query planning - too many subqueries or query is too complex.

(job ID: f884c149-c3fd-4a1e-b935-6b45ec7dd3da)

                                                           -----Query Job SQL Follows-----                                                           

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:-- All urine culture orders at Stanford, order_id must be in lab_results
   2:WITH urine_cultures AS (
   3:    SELECT DISTINCT
   4:        op.anon_id, op.pat_enc_csn_id_coded, op.order_proc_id_coded, op.order_time_jittered_utc, op.ordering_mode
   5:    FROM 
   6:        `mining-clinical-decisions.shc_core.order_proc` op
   7:    INNER JOIN
   8:        `mining-clinical-decisions.shc_core.lab_result` lr
   9:    ON
  10:        op.order_proc_id_coded = lr.order_id_coded
  11:    WHERE
  12:        

In [60]:
print(df_uc_all.shape)
df_uc_all.head(5)

(280841, 11)


,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,ordering_mode,was_positive,had_other_pos_culture,ordered_antibiotics,organisms,other_pos_sites,other_organisms
0,JC29fce55,1952602,303563619,2007-12-14 16:22:00+00:00,Outpatient,1,0,None,ESCHERICHIA COLI,None,None
1,JC29fce55,1952602,303563620,2007-12-14 16:22:00+00:00,Outpatient,1,0,None,ESCHERICHIA COLI,None,None
2,JCd247d0,6619001,303567177,2003-03-14 20:13:00+00:00,Outpatient,0,0,None,None,None,None
3,JCddffb8,14533332,303568988,2000-08-16 16:33:00+00:00,Outpatient,0,0,None,None,None,None
4,JCd3f4b3,6619302,303572415,2001-09-13 18:30:00+00:00,Outpatient,0,0,None,None,None,None


In [79]:
%%bigquery df_prior_abx
-- Assuming above query has been saved to mining-clinical-decisions.conor_db.uti_cohort
WITH cohort AS (
    SELECT * FROM mining-clinical-decisions.conor_db.uti_cohort
),

-- Set of antibiotics that show up in order_med (may not be totally all encompassing) 
include_abx AS (
    SELECT
        med_description
    FROM
        `mining-clinical-decisions.abx.abx_types` 
    WHERE
        is_include_abx = 1 OR is_oral = 1
),

prior_abx AS (
    SELECT DISTINCT
        cohort.order_proc_id_coded
    FROM 
        cohort
    INNER JOIN
        `mining-clinical-decisions.shc_core.order_med` om
    USING
        (anon_id)
    WHERE
        om.med_description IN (SELECT med_description FROM include_abx)
    AND 
        TIMESTAMP_DIFF(om.order_inst_utc, cohort.order_time_jittered_utc, HOUR) BETWEEN -24*14 AND 0 
)

-- Remove observations with prior antibiotics
SELECT DISTINCT
    *
FROM 
    cohort
WHERE 
    order_proc_id_coded NOT IN (SELECT DISTINCT order_proc_id_coded FROM prior_abx)


In [80]:
print(df_prior_abx.shape)
df_prior_abx.head()

(177977, 11)


,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,ordering_mode,was_positive,had_other_pos_culture,ordered_antibiotics,organisms,other_pos_sites,other_organisms
0,JC29fce55,1952602,303563619,2007-12-14 16:22:00+00:00,Outpatient,1,0,None,ESCHERICHIA COLI,None,None
1,JC29fce55,1952602,303563620,2007-12-14 16:22:00+00:00,Outpatient,1,0,None,ESCHERICHIA COLI,None,None
2,JCd247d0,6619001,303567177,2003-03-14 20:13:00+00:00,Outpatient,0,0,None,None,None,None
3,JCddffb8,14533332,303568988,2000-08-16 16:33:00+00:00,Outpatient,0,0,None,None,None,None
4,JCd3f4b3,6619302,303572415,2001-09-13 18:30:00+00:00,Outpatient,0,0,None,None,None,None


In [90]:
def string_agg_cols(arr):
    final_set = []
    for a in arr:
        if a is None:
            continue
        groups = a.split(',')
        for g in groups:
            if g not in final_set:
                final_set.append(g)
    final_set.sort()
    if len(final_set) == 0:
        return None
    else:
        return ','.join([f for f in final_set])
    
groupby_cols = ['anon_id', 'pat_enc_csn_id_coded', 'order_time_jittered_utc']
df_final = (df_prior_abx
    .groupby(groupby_cols)
    .agg({'order_proc_id_coded' : 'first',
          'ordering_mode' : 'first',
          'was_positive' : 'max',
          'had_other_pos_culture' : 'max',
          'ordered_antibiotics' : string_agg_cols,
          'organisms' : string_agg_cols,
          'other_pos_sites' : string_agg_cols,
          'other_organisms' : string_agg_cols})
    .reset_index()
)
print(df_final.shape)
df_final.head()

(174640, 11)


,anon_id,pat_enc_csn_id_coded,order_time_jittered_utc,order_proc_id_coded,ordering_mode,was_positive,had_other_pos_culture,ordered_antibiotics,organisms,other_pos_sites,other_organisms
0,JC29f8aa6,131284475290,2020-03-19 23:33:00+00:00,659051139,Outpatient,0,0,None,None,None,None
1,JC29f8ae9,131267983420,2019-07-02 22:10:00+00:00,615279161,Outpatient,0,0,None,None,None,None
2,JC29f8b0f,131280361737,2019-11-19 08:00:00+00:00,646120545,Outpatient,0,0,None,None,None,None
3,JC29f8b1b,131264672614,2019-02-20 08:00:00+00:00,601143909,Outpatient,0,0,None,None,None,None
4,JC29f8b1c,131284098448,2020-02-18 21:48:00+00:00,657689560,Outpatient,0,0,None,None,None,None


In [104]:
df_final.to_gbq(
    destination_table='conor_db.uti_cohort_final',
    project_id='mining-clinical-decisions',
    if_exists='replace'
)


/Users/conorcorbin/opt/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
1it [00:35, 35.24s/it]


In [91]:
df_uc_all = df_final

In [92]:
df_uc_all = df_uc_all[~df_uc_all.duplicated(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_time_jittered_utc'])]
df_uc_all.shape

(174640, 11)

### Inpatient vs outpatient

In [93]:
df_uc_all.ordering_mode.value_counts(dropna=False)

Outpatient    105594
Inpatient      65495
NaN             3551
Name: ordering_mode, dtype: int64

### Positive vs negative cultures

In [94]:
df_uc_all.was_positive.value_counts()

0    140620
1     34020
Name: was_positive, dtype: int64

### Positive vs negative by ordering mode

In [95]:
print("Outpatient positivity rate")
print(df_uc_all.query('ordering_mode == "Outpatient"', engine='python')['was_positive'].mean())
print("Inpatient positivity rate")
print(df_uc_all.query('ordering_mode == "Inpatient"', engine='python')['was_positive'].mean())


Outpatient positivity rate
0.1488626247703468
Inpatient positivity rate
0.2581113062065807


### How many had other positive cultures at different anatomical sites?

In [96]:
df_uc_all.had_other_pos_culture.value_counts()

0    172186
1      2454
Name: had_other_pos_culture, dtype: int64

### Repeat within positive and negative urine cultures

In [97]:
df_uc_all[['was_positive', 'had_other_pos_culture']].value_counts()

was_positive  had_other_pos_culture
0             0                        139100
1             0                         33086
0             1                          1520
1             1                           934
dtype: int64

### Of all cases where urine culture was negative but another culture was positive, what were most common anatomical sites of the positive cultures?


In [98]:
(df_uc_all
.query('was_positive == 0 and had_other_pos_culture == 1', engine='python')
['other_pos_sites']
).value_counts()


URINE CULTURE                                                               262
BLOOD CULTURE (AEROBIC & ANAEROBIC BOTTLES)                                 243
BLOOD CULTURE (2 AEROBIC BOTTLES)                                           160
RESPIRATORY CULTURE                                                         123
WOUND CULTURE                                                               107
                                                                           ... 
ORTHOPEDIC AEROBIC CULT W/ GRAM STAIN                                         1
 DEEP / SURGICAL / ASP,WOUND CULTURE,WOUND CULTURE AND GRAM STAIN             1
 NON - RESPIRATORY,AFB CULTURE                                                1
BLOOD CULTURE (2 AEROBIC BOTTLES),FUNGAL CULTURE                              1
BLOOD CULTURE (AEROBIC & ANAEROBIC BOTTLES),ORTHOPEDIC AEROBIC CULT W/GS      1
Name: other_pos_sites, Length: 85, dtype: int64

### Most common antibiotic orders

In [99]:
df_uc_all.ordered_antibiotics.value_counts(dropna=False).head(20)

NaN                                                                        151248
NITROFURANTOIN MONOHYD/M-CRYST 100 MG PO CAPS                                2653
CIPROFLOXACIN 500 MG PO TABS                                                 1223
SULFAMETHOXAZOLE-TRIMETHOPRIM 800-160 MG PO TABS                             1181
CIPROFLOXACIN HCL 500 MG PO TABS                                             1148
CEPHALEXIN 500 MG PO CAPS                                                    1148
CIPROFLOXACIN HCL 250 MG PO TABS                                              371
CIPROFLOXACIN 250 MG PO TABS                                                  332
CIPROFLOXACIN IN D5W 400 MG/200 ML IV PGBK                                    298
CEFAZOLIN IN DEXTROSE (ISO-OS) 1 GRAM/50 ML IV PGBK                           248
PIPERACILLIN-TAZOBACTAM-DEXTRS 3.375 GRAM/50 ML IV PGBK                       242
CEFTRIAXONE 1 GRAM/10 ML IV PUSH (VIAL ONLY)                                  238
LEVOFLOXACIN 500

In [100]:
df_uc_all['order_proc_id_coded'].nunique()

174640

### Most common organisms

In [101]:
df_uc_all.organisms.value_counts(dropna=False).head(20)

NaN                                            140620
ESCHERICHIA COLI                                18046
ENTEROCOCCUS SPECIES                             3476
KLEBSIELLA PNEUMONIAE                            2686
STREPTOCOCCUS AGALACTIAE (GROUP B)               1211
ENTEROCOCCUS SPECIES,ESCHERICHIA COLI            1080
PROTEUS MIRABILIS                                1015
STREPTOCOCCUS AGALACTIAE {GROUP B}                976
PSEUDOMONAS AERUGINOSA                            486
KLEBSIELLA OXYTOCA                                347
ESCHERICHIA COLI,KLEBSIELLA PNEUMONIAE            338
ZZZENTEROBACTER AEROGENES                         252
ENTEROBACTER CLOACAE COMPLEX                      238
ENTEROCOCCUS FAECALIS                             212
STAPHYLOCOCCUS AUREUS                             182
CITROBACTER KOSERI                                172
CITROBACTER FREUNDII COMPLEX                      158
ESCHERICHIA COLI,PROTEUS MIRABILIS                153
CITROBACTER KOSERI (CARBAPEN

### Cultures by year

In [102]:
df_uc_all.assign(year=lambda x: x.order_time_jittered_utc.dt.year)[['year']].value_counts()

year
2018    21686
2019    21272
2017    19523
2016    17986
2015    13183
2013     9458
2009     9287
2008     9248
2014     9011
2011     8945
2010     8789
2012     8776
2020     4660
2000     2310
2001     2237
2002     1804
2003     1742
2004     1650
2005     1581
2006     1490
2007        2
dtype: int64

In [103]:
df_uc_all[['anon_id', 'pat_enc_csn_id_coded', 'order_time_jittered_utc']].duplicated

<bound method DataFrame.duplicated of           anon_id  pat_enc_csn_id_coded   order_time_jittered_utc
0       JC29f8aa6          131284475290 2020-03-19 23:33:00+00:00
1       JC29f8ae9          131267983420 2019-07-02 22:10:00+00:00
2       JC29f8b0f          131280361737 2019-11-19 08:00:00+00:00
3       JC29f8b1b          131264672614 2019-02-20 08:00:00+00:00
4       JC29f8b1c          131284098448 2020-02-18 21:48:00+00:00
...           ...                   ...                       ...
174635   JCec48e9          131250306582 2018-04-07 04:42:00+00:00
174636   JCec48f2          131231090378 2017-05-07 07:00:00+00:00
174637   JCec48fd          131226848112 2017-05-05 22:24:00+00:00
174638   JCec48fe          131236053941 2017-07-17 23:55:00+00:00
174639   JCec48fe          131242938601 2018-01-15 06:07:00+00:00

[174640 rows x 3 columns]>

In [37]:
df_uc_all[df_uc_all.duplicated(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_time_jittered_utc'], keep=False)].sort_values('pat_enc_csn_id_coded').tail(20)


,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,ordering_mode,was_positive,had_other_pos_culture,ordered_antibiotics,organisms,other_pos_sites,other_organisms
268156,JCd3b903,131276079369,640785148,2019-10-13 19:36:00+00:00,Outpatient,1,0,NITROFURANTOIN MONOHYD/M-CRYST 100 MG PO CAPS,KLEBSIELLA PNEUMONIAE,None,None
268171,JCd3b903,131276079369,640799200,2019-10-13 19:36:00+00:00,Outpatient,0,0,NITROFURANTOIN MONOHYD/M-CRYST 100 MG PO CAPS,None,None,None
263832,JCe68799,131276290589,633855188,2019-08-30 17:30:00+00:00,Inpatient,1,0,None,ESCHERICHIA COLI,None,None
263831,JCe68799,131276290589,633855185,2019-08-30 17:30:00+00:00,Inpatient,0,0,None,None,None,None
264084,JCe4335a,131276471298,634290860,2019-08-31 18:28:00+00:00,Outpatient,0,0,None,None,None,None
264085,JCe4335a,131276471298,634290871,2019-08-31 18:28:00+00:00,Outpatient,0,0,None,None,None,None
267222,JCe0c742,131278280904,639331707,2019-10-27 18:16:00+00:00,Outpatient,0,0,None,None,None,None
267223,JCe0c742,131278280904,639331708,2019-10-27 18:16:00+00:00,Outpatient,1,0,None,STAPHYLOCOCCUS AUREUS,None,None
274705,JCd48cc2,131279918491,650891865,2020-01-27 23:54:00+00:00,Outpatient,1,1,AMOXICILLIN-POT CLAVULANATE 875-125 MG PO TABS,PSEUDOMONAS AERUGINOSA,WOUND CULTURE AND GRAM STAIN,"PSEUDOMONAS AERUGINOSA,STAPH AUREUS {MRSA}"
274706,JCd48cc2,131279918491,650891866,2020-01-27 23:54:00+00:00,Outpatient,1,1,AMOXICILLIN-POT CLAVULANATE 875-125 MG PO TABS,PSEUDOMONAS AERUGINOSA,WOUND CULTURE AND GRAM STAIN,"PSEUDOMONAS AERUGINOSA,STAPH AUREUS {MRSA}"
